In [1]:
import cv2
import numpy as np
import os
import glob

In [2]:
# Defining the dimensions of checkerboard(長寬各有多少角點數)
CHECKERBOARD = (6,9)

# 設置像素角點的參數，停止準則採用最大循環次數(max iter)+最大容忍誤差(eps)為30.00
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Creating vector to store vectors of 3D points for each checkerboard image(world)
objpoints = [] # store 3D points
# Creating vector to store vectors of 2D points for each checkerboard image(image)
imgpoints = [] # store 2D points

# Defining the world coordinates for 3D points(定義世界座標=>描述物體真實的位置)
## 產生三維的零矩陣
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
## 將世界坐標系建在標定版上，所有的Z點座標都為0，所以只有X、Y需要給予數值
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [3]:
# Extracting path of individual image stored in a given directory
images = glob.glob('./image/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    # 轉換成灰階
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        ## 在原角點的基礎上尋找亞像素角點
        # cornerSubPix(image,corners,winSize zeroZone)
        '''
        winSize = 區域大小為 NXN; N=(winSize*2+1) (計算亞畫素角點時考慮的區域的大小)
        zeroZone =  類似於winSize，但是總具有較小的範圍，Size(-1,-1)表示忽略
        '''
        corners2 = cv2.cornerSubPix(gray, corners, (9,9),(-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.imshow('img',img)
    cv2.waitKey(1500)

cv2.destroyAllWindows()

h,w = img.shape[:2]

"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix(intrinsic parameter) : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs(extrinsic parameter) : \n")
print(rvecs)
print("tvecs(extrinsic parameter) : \n")
print(tvecs)

Camera matrix(intrinsic parameter) : 

[[3.03282800e+03 0.00000000e+00 1.53937659e+03]
 [0.00000000e+00 3.03018944e+03 2.01631367e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[ 0.03258498 -0.21949084  0.00047211  0.0051064   0.35615065]]
rvecs(extrinsic parameter) : 

(array([[ 0.03870172],
       [-0.14377813],
       [-0.09575442]]), array([[ 0.1225424 ],
       [-0.20455739],
       [-0.07649326]]), array([[ 0.20910126],
       [-0.17313041],
       [-0.07735994]]), array([[ 0.34729888],
       [-0.19311932],
       [-0.02122464]]), array([[ 0.37139541],
       [-0.19295209],
       [ 0.03791292]]), array([[ 0.46245695],
       [-0.20904842],
       [ 0.01759843]]), array([[ 0.51599737],
       [-0.14722798],
       [ 0.0692641 ]]), array([[-0.21550322],
       [-0.83552028],
       [-2.97212708]]), array([[-0.22584747],
       [-0.82121307],
       [-2.98892061]]), array([[-0.22791251],
       [-0.81032352],
       [-2.9831623 ]]), array([[-0.21393876],
       [-

In [4]:
print("Camera matrix(intrinsic parameter) : \n")
print(mtx)

Camera matrix(intrinsic parameter) : 

[[3.03209507e+03 0.00000000e+00 1.53957273e+03]
 [0.00000000e+00 3.02940044e+03 2.01690607e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [9]:
print("rvecs(extrinsic parameter - rotation) : \n")
print(rvecs)

rvecs(extrinsic parameter - rotation) : 

(array([[ 0.03874374],
       [-0.1436753 ],
       [-0.09578558]]), array([[ 0.12259614],
       [-0.20454763],
       [-0.07648667]]), array([[ 0.20925306],
       [-0.17351986],
       [-0.0773406 ]]), array([[ 0.3483956 ],
       [-0.19293506],
       [-0.02102626]]), array([[ 0.37138768],
       [-0.1931297 ],
       [ 0.03787842]]), array([[ 0.46242887],
       [-0.20958239],
       [ 0.01747835]]), array([[ 0.51611731],
       [-0.1471601 ],
       [ 0.06928937]]), array([[-0.21552364],
       [-0.83509319],
       [-2.97228608]]), array([[-0.22583425],
       [-0.82081788],
       [-2.98905375]]), array([[-0.22783535],
       [-0.80992324],
       [-2.98328722]]), array([[-0.21328112],
       [-0.8103009 ],
       [-3.00533065]]), array([[-0.08425705],
       [-0.50705175],
       [-3.07779071]]), array([[0.14403834],
       [0.36929905],
       [3.02059991]]), array([[0.18459827],
       [0.19092788],
       [2.99316048]]), array([[ 0.

In [10]:
print("tvecs(extrinsic parameter) : \n")
print(tvecs)

tvecs(extrinsic parameter) : 

(array([[-1.92211326],
       [-3.82593529],
       [14.26286739]]), array([[-2.98723997],
       [-4.33932751],
       [13.2419361 ]]), array([[-1.32488614],
       [-3.46491686],
       [11.44745228]]), array([[-2.10795316],
       [-3.64949385],
       [11.33966254]]), array([[-2.36125968],
       [-3.0911657 ],
       [11.47370388]]), array([[-2.8611693 ],
       [-2.57417679],
       [11.02078557]]), array([[-1.81899192],
       [-1.59081133],
       [10.24757616]]), array([[ 1.93447403],
       [ 2.22377334],
       [10.07732881]]), array([[ 2.10268096],
       [ 2.29120145],
       [10.15736653]]), array([[ 2.06375868],
       [ 2.23266564],
       [10.18652308]]), array([[ 1.97523159],
       [ 2.53590754],
       [10.12808656]]), array([[ 1.37210998],
       [ 1.62934138],
       [11.29091168]]), array([[ 2.88259444],
       [ 3.92034954],
       [12.47857623]]), array([[ 3.2772662 ],
       [ 3.93565122],
       [13.79035067]]), array([[ 2.99352